In [1]:
# Imports
import os
import re
import cv2
import json
import argparse
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [137]:
# Config
cnn_color     =  (0,255,0)
opencv_color  =  (0,0,255)   
tracker_color =  (255,0,0)
HRATIO        =  1#frame.shape[0] / HEIGHT
WRATIO        =  1#frame.shape[1] / WIDTH
fourcc        =  cv2.VideoWriter_fourcc(*'XVID')

# parser = argparse.ArgumentParser()
# parser.add_argument("--name")
# value = parser.parse_args()


# name = files.split('.')[0]
name = "00b5a2d0-8574-4ae7-9147-fb64f085c295"
video_path = 'input/'+name+'.mp4'
json_path  = 'jsons/'+name+'.json'

f = open(json_path).readlines()
data = json.loads('{'+ f[0][10:-2])
# data = json.load(f)
new_df = pd.DataFrame(columns=['FRAME_COUNT', 'OPEN_BOX', 'CNN_BOX','Vertical', 'SPEED', 'POSE', 'SUI', 'SPLASH', 'ACTIND', 'TBBOX', 'ALARM'])



In [136]:
df = pd.read_csv('test.csv')
rr = df[9:10]
tt = rr['Additional Info'].values[0]
print("Video Name : ", rr['Video Link'].values)

Video Name :  ['https://camerye-cloudserver.s3-us-west-2.amazonaws.com/assets/00:04:4b:ea:53:00/events/4283205.mp4']


In [141]:
!wget https://camerye-cloudserver.s3-us-west-2.amazonaws.com/assets/00:04:4b:ea:53:00/events/4283205.mp4

--2022-03-21 15:48:47--  https://camerye-cloudserver.s3-us-west-2.amazonaws.com/assets/00:04:4b:ea:53:00/events/4283205.mp4
Resolving camerye-cloudserver.s3-us-west-2.amazonaws.com (camerye-cloudserver.s3-us-west-2.amazonaws.com)... 52.218.153.233
Connecting to camerye-cloudserver.s3-us-west-2.amazonaws.com (camerye-cloudserver.s3-us-west-2.amazonaws.com)|52.218.153.233|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7482157 (7.1M) [video/mp4]
Saving to: ‘4283205.mp4’

4283205.mp4         100%[===================>]   7.13M  34.7KB/s    in 2m 1s   

2022-03-21 15:50:50 (60.3 KB/s) - ‘4283205.mp4’ saved [7482157/7482157]



In [138]:
data = json.loads('{'+ [tt][0][10:-2]) 

In [139]:
ind = 0
for key in data.keys():
    D = data[key]
    T = json.loads(D['TRACKERS_DATA'])
    for key in list(T.keys()):
        T = json.loads(D['TRACKERS_DATA'])[key]
        new_df.loc[ind] = [str(D['FRAME_COUNT'])] +  [str(D['OPEN_BOX'])] + [str( D['CNN_BOX'])] + [str(T['VERTICAL'])] + [str(T['SPEED'])] + [str(T['POSE'])] + \
                        [str(T['SUI'])] + [str(T['SPLASH'])] + [str(T['ACTIND'])] + [str(T['BBOX'])] + [str(T['ALARM'])]

        ind += 1
    if len(T.keys())==0:
        new_df.loc[ind] = [str(D['FRAME_COUNT'])] +  [str(D['OPEN_BOX'])] + [str( D['CNN_BOX'])]\
                        + [""] + [""] +  [""]  +  [""] + [""] + [""] + [""]+ [""]
        ind += 1
        
del ind

In [140]:
new_df[100:200]

,FRAME_COUNT,OPEN_BOX,CNN_BOX,Vertical,SPEED,POSE,SUI,SPLASH,ACTIND,TBBOX,ALARM
100,387078,[],[],,,,,,,,
101,387079,[],[],,,,,,,,
102,387080,[],[],,,,,,,,
103,387081,[],[],,,,,,,,
104,387082,[],[],,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...
195,387173,[],"[[1153.0, 245.875, 1276.0, 446.5]]",31.96562339232915,0.16539715120408047,1.381642512077295,0.8482905982905985,34.71,0.0,"[256, 513, 542, 720]",False
196,387174,[],"[[1154.0, 227.75, 1275.0, 438.75]]",31.96562339232915,0.13231772096326438,1.381642512077295,0.9205128205128205,34.865,0.0,"[256, 513, 542, 720]",False
197,387175,[],[],31.969233557063145,0.10585417677061151,1.381642512077295,0.918948283355063,34.82,0.0,"[256, 513, 542, 720]",False
198,387175,[],[],0.0,0.0,0.5528846153846154,0.13169104074968052,0.35,239.20000000000002,"[1163, 231, 1278, 439]",False


In [142]:
THRESHOLD = 100
new_df['Vertical'] = pd.to_numeric(new_df['Vertical']).clip(upper=THRESHOLD)
new_df['SPEED']    = pd.to_numeric(new_df['SPEED']).clip(upper=THRESHOLD)
new_df['POSE_50']  = pd.to_numeric(new_df['POSE']) * 50 
new_df['SUI_50']   = pd.to_numeric(new_df['SUI']) * 50
new_df['SPLASH']   = pd.to_numeric(new_df['SPLASH']).clip(upper=THRESHOLD)
new_df['ACTIND']   = pd.to_numeric(new_df['ACTIND']).clip(upper=THRESHOLD)

In [143]:
new_df = new_df.reset_index()

In [144]:
# new_df[100:200]

In [145]:
# rm  AnalysisOutput/*.mp4

In [125]:
frame.shape

(768, 1280, 3)

In [146]:
video = cv2.VideoCapture('4283205.mp4')
analysis_feed  = cv2.VideoWriter("AnalysisOutput/4283205.mp4",fourcc, 5, (1280,720))
FRAME_COUNT = 0
try:
    while video.isOpened():
        print(FRAME_COUNT)
        FRAME_COUNT += 1
        ret_val, frame = video.read()
        centroid = None
        ## TRACKER 
        arr = new_df.iloc[FRAME_COUNT, 10]
        NO_OF_TRACKER = 0
        if len(arr)>3:
            temp = re.findall(r'\d+', arr)
            temp = [t for t in temp if len(t)<=4]
            temp = list(map(int, temp))
            NO_OF_TRACKER = len(temp)//4
            for ind in range(len(temp)//4):
                ind         = ind * 4
                bbox        = temp[ind] , temp[ind+1] , temp[ind+2] , temp[ind+3] 
                start_point = ( int(bbox[0]*WRATIO ) , int(bbox[1] * HRATIO)  )
                end_point   = (int(bbox[2]*WRATIO), int(bbox[3]* HRATIO ) )
                image       = cv2.rectangle(frame, start_point, end_point, tracker_color, 1)
                centroid    = ( int(bbox[0] +(bbox[2]-bbox[0])/2), int(bbox[1] + (bbox[3]-bbox[1])/2))
                centroid    = (int(centroid[0] * HRATIO) , int(centroid[1] * WRATIO))
                frame       = cv2.circle(frame, centroid, 5, tracker_color, 2)


        cv2.putText(frame,"No of person: " + str(NO_OF_TRACKER), (50,150), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,0,255),1,1)
        analysis_feed.write(frame)

except Exception as e:
    print("ERROR ", e)
    analysis_feed.release()
analysis_feed.release()



OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
ERROR  single positional index

In [124]:
analysis_feed.release()